#1° Problema de Negócio
Precisamos construir uma Máquina Preditiva que, para ser usada na classificação de uma pessoa para participar ou de um programa de intercâmbio.


Fonte de Dados: UCI - Adults
http://archive.ics.uci.edu/ml/datasets/Adult

**Atualizações**

In [ ]:
#Comando de Instalação do Algoritmo
!pip install lightgbm

In [ ]:
#Comando de Instalação do Algoritmo
!pip install xgboost

In [3]:
# Instalar para tratar o warning  warnings.warn(msg, FutureWarning) no import das bibliotecas
!pip install "dask[dataframe]"

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.6 MB/s eta 0:00:00


In [23]:
# Importando as bibliotecas
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, roc_auc_score
from datetime import datetime
import matplotlib.patches as mpatches

In [2]:
# Importação dos dados a serem utilizados para a criação do modelo de ML
#dados = pd.read_csv('https://raw.githubusercontent.com/llSourcell/Best-Programming-Languages-for-Machine-Learning/master/adults.txt', sep=',')

colunas = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
dados = pd.read_csv('adult.data', names=colunas, sep=', ', engine='python')

dados

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


# 2°  Análise Exploratória

# A ANÁLISE DETALHADA DOS DADOS ESTÃO NO ARQUIVO **ANALISE_DE_DADOS.IPYNB**

In [4]:
#glimpse of the dataset
dados.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [6]:
dados.duplicated().sum()

24

#3° Pré-Processamento dos Dados

In [3]:
#Eliminando as 24 linhas duplicadas
dados.drop_duplicates(keep='first', inplace=True)
dados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32537 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32537 non-null  int64 
 1   workclass       32537 non-null  object
 2   fnlwgt          32537 non-null  int64 
 3   education       32537 non-null  object
 4   education-num   32537 non-null  int64 
 5   marital-status  32537 non-null  object
 6   occupation      32537 non-null  object
 7   relationship    32537 non-null  object
 8   race            32537 non-null  object
 9   sex             32537 non-null  object
 10  capital-gain    32537 non-null  int64 
 11  capital-loss    32537 non-null  int64 
 12  hours-per-week  32537 non-null  int64 
 13  native-country  32537 non-null  object
 14  salary          32537 non-null  object
dtypes: int64(6), object(9)
memory usage: 4.0+ MB


In [3]:
# Lista das colunas a serem excluídas
colunas_para_excluir = ['age', 'fnlwgt', 'education-num', 'relationship', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']

# Excluindo as colunas do DataFrame
dados = dados.drop(columns=colunas_para_excluir)

In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   workclass       32561 non-null  object
 1   education       32561 non-null  object
 2   marital-status  32561 non-null  object
 3   occupation      32561 non-null  object
 4   race            32561 non-null  object
 5   sex             32561 non-null  object
 6   salary          32561 non-null  object
dtypes: object(7)
memory usage: 1.7+ MB


In [5]:
# Para verificar a quantidade de ?
dados['workclass'].value_counts()

,count
workclass,
Private,22696
Self-emp-not-inc,2541
Local-gov,2093
?,1836
State-gov,1298
Self-emp-inc,1116
Federal-gov,960
Without-pay,14
Never-worked,7


In [6]:
# Calcula a moda da coluna 'workclass' para "distribuir" esses valores
moda = dados['workclass'].mode()[0]

# Substitui os valores "?" pela moda
dados['workclass'] = dados['workclass'].replace('?', moda)

In [27]:
# Unir valores de 'Without-pay' para 'Never-worked'
workclass_mapping = {
    'Never-worked': 'Without-pay', # Mapeia 'Never-worked' para 'Without-pay'
}

# Aplica a substituição na coluna 'workclass'
dados['workclass'] = dados['workclass'].replace(workclass_mapping)

In [29]:
# Para verificar a quantidade de ?
dados['workclass'].value_counts()

,count
workclass,
Private,24532
Self-emp-not-inc,2541
Local-gov,2093
State-gov,1298
Self-emp-inc,1116
Federal-gov,960
Without-pay,21


In [7]:
# Para verificar a quantidade de ?
dados['occupation'].value_counts()

,count
occupation,
Prof-specialty,4140
Craft-repair,4099
Exec-managerial,4066
Adm-clerical,3770
Sales,3650
Other-service,3295
Machine-op-inspct,2002
?,1843
Transport-moving,1597


In [8]:
# Substitui os dados com '?' em 'occupation' por 'other-service'
dados['occupation'] = dados['occupation'].replace('?', 'Other-service')

In [9]:
# Para verificar a quantidade de ?
dados['occupation'].value_counts()

,count
occupation,
Other-service,5138
Prof-specialty,4140
Craft-repair,4099
Exec-managerial,4066
Adm-clerical,3770
Sales,3650
Machine-op-inspct,2002
Transport-moving,1597
Handlers-cleaners,1370


In [11]:
# Para verificar os "tipos de casados"
dados['marital-status'].value_counts()

,count
marital-status,
Married-civ-spouse,14976
Never-married,10683
Divorced,4443
Separated,1025
Widowed,993
Married-spouse-absent,418
Married-AF-spouse,23


In [12]:
# Unificar os tipos de casados
dados['marital-status'] = dados['marital-status'].replace(['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse'], 'Married')

# Unir os never-married com widowed
dados['marital-status'] = dados['marital-status'].replace(['Never-married'], 'Widowed')


In [13]:
# Para verificar os "tipos de casados"
dados['marital-status'].value_counts()

,count
marital-status,
Married,15417
Widowed,11676
Divorced,4443
Separated,1025


In [15]:
# Para verificar os niveis escolar
dados['education'].value_counts()

,count
education,
HS-grad,10501
Some-college,7291
Bachelors,5355
Masters,1723
Assoc-voc,1382
11th,1175
Assoc-acdm,1067
10th,933
7th-8th,646


In [ ]:
# Unindo os graus incompletos em some-college
education_mapping = {
    '11th': 'Some-college',
    '10th': 'Some-college',
    '7th-8th': 'Some-college',
    '9th': 'Some-college',
    '12th': 'Some-college',
    '5th-6th': 'Some-college',
    '1st-4th': 'Some-college',
    'Preschool': 'Some-college'
}

# Substitui os valores na coluna "education"
dados['education'] = dados['education'].replace(education_mapping)

In [18]:
# Para verificar os niveis escolar
dados['education'].value_counts()

,count
education,
Some-college,11544
HS-grad,10501
Bachelors,5355
Masters,1723
Assoc-voc,1382
Assoc-acdm,1067
Prof-school,576
Doctorate,413


In [19]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   workclass       32561 non-null  object
 1   education       32561 non-null  object
 2   marital-status  32561 non-null  object
 3   occupation      32561 non-null  object
 4   race            32561 non-null  object
 5   sex             32561 non-null  object
 6   salary          32561 non-null  object
dtypes: object(7)
memory usage: 1.7+ MB


In [21]:
# Define as cores para as fatias do gráfico (ajuste conforme necessário)
colors = ['lightblue', 'lightcoral', 'lightgreen', 'lightskyblue', 'lightpink', 'lightyellow', 'orange', 'gray']
sns.set_theme(style="whitegrid")

In [26]:
# Para verificar os niveis escolar
dados['workclass'].value_counts()

,count
workclass,
Private,24532
Self-emp-not-inc,2541
Local-gov,2093
State-gov,1298
Self-emp-inc,1116
Federal-gov,960
Without-pay,14
Never-worked,7


In [36]:
# Usando o OneHor Encoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

colunas_cat = ['workclass', 'education', 'marital-status', 'occupation', 'race', 'sex']

encoder.fit(dados[colunas_cat])

encoded_dados = encoder.transform(dados[colunas_cat])
encoded_df = pd.DataFrame(encoded_dados, columns=encoder.get_feature_names_out(colunas_cat))

dados = pd.concat([dados, encoded_df], axis=1)
dados = dados.drop(columns=colunas_cat)

In [38]:
#Comparando o nome das colunas para ver se existe alguma duplicada
colunas_vistas = set()
colunas_duplicadas = []

for coluna in dados.columns:
    if coluna in colunas_vistas:
        colunas_duplicadas.append(coluna)
    else:
        colunas_vistas.add(coluna)

if colunas_duplicadas:
    print("Colunas duplicadas encontradas:", colunas_duplicadas)
else:
    print("Não foram encontradas colunas duplicadas.")

Não foram encontradas colunas duplicadas.


In [39]:
salary_counts = dados.groupby('salary')['salary'].count()

# Exibe os resultados
print(salary_counts)

salary
<=50K    24720
>50K      7841
Name: salary, dtype: int64


In [40]:
# Transformando a variavel target que <=50K e >50 em 0 e 1
le=LabelEncoder()
le.fit(dados.salary)

le.classes_
dados.salary=Series(le.transform(dados.salary))  #Aqui o labelEncoder substiyui os dados
dados.salary.value_counts()
dados.salary

,salary
0,0
1,0
2,0
3,0
4,0
...,...
32556,0
32557,1
32558,0
32559,0


In [42]:
# Agrupa os itens da coluna 'salary' e calcula o total em cada grupo
total_por_salary = dados.groupby('salary').size()

# Exibe o total por grupo
print(total_por_salary)

salary
0    24720
1     7841
dtype: int64


In [44]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   salary                           32561 non-null  int64  
 1   workclass_Autônomo com trabalho  32561 non-null  float64
 2   workclass_Autônomo sem trabalho  32561 non-null  float64
 3   workclass_Desempregado           32561 non-null  float64
 4   workclass_Governo Estadual       32561 non-null  float64
 5   workclass_Governo Federal        32561 non-null  float64
 6   workclass_Governo Local          32561 non-null  float64
 7   workclass_Privado                32561 non-null  float64
 8   education_Assoc-acdm             32561 non-null  float64
 9   education_Assoc-voc              32561 non-null  float64
 10  education_Bachelors              32561 non-null  float64
 11  education_Doctorate              32561 non-null  float64
 12  education_HS-grad 

In [45]:
# Separando X e Y
x = dados.drop('salary', axis=1)
y = dados['salary']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3)

y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 32561 entries, 0 to 32560
Series name: salary
Non-Null Count  Dtype
--------------  -----
32561 non-null  int64
dtypes: int64(1)
memory usage: 254.5 KB


#3° Máquina Preditiva

## XGBoost

In [46]:
#Criando o dataset de treino e teste do XGB
dtrain_xgb = xgb.DMatrix(x_train,label=y_train)
dtest_xgb = xgb.DMatrix(x_test)

In [47]:
#Setando os parâmetros do XGBoost
parameters_xgb = {'max_depth':5,
              'eta':1,
              'silent':1,
              'objective':'binary:logistic',
              'eval_metric':'auc','learning_rate':.05}

# Treinando o modelo
num_round = 50
start = datetime.now() # para verificar o tempo de execução

# Treinando o modelo
xg = xgb.train(parameters_xgb, dtrain_xgb, num_round)

stop = datetime.now()   # Tempo de execução dos modelos
execution_time_xgb = stop-start
execution_time_xgb
print("Tempo de execução: ", execution_time_xgb)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:04:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Tempo de execução:  0:00:00.456402


In [48]:
# Predição do teste
ypred_xgb = xg.predict(dtest_xgb)
ypred_xgb.size

ypred_xgb[0:5]  # apresentando as 5 primeiras predições

array([0.0446732 , 0.04350259, 0.4144815 , 0.37347656, 0.5606989 ],
      dtype=float32)

In [49]:
# Converte a probabilidade entre 1 ou 0
ypred_xgb = (ypred_xgb >= 0.5).astype(int) # Altere esta linha para garantir que ypred é uma matriz inteira

In [50]:
# Avaliando a Máquina Preditiva XGBoost
accuracy_xgb = accuracy_score(ypred_xgb, y_test)
accuracy_xgb

print(f"Acurácia: {accuracy_xgb * 100:.2f}%")

Acurácia: 82.30%


In [51]:
y_test.value_counts()

,count
salary,
0,7415
1,2354


In [52]:
# roc_auc_score (taxa de verdadeiros positivos (TPR) contra a taxa de falsos positivos (FPR)) para xgboost
# Calculando Modelo XGB
auc_xgb = roc_auc_score(y_test, ypred_xgb)
auc_xgb

print(f"ROC Acurácia: {auc_xgb * 100:.2f}%")

ROC Acurácia: 69.92%


## Light_GBM

In [53]:
# Criando o dataset de treinamento
train_data_lgb = lgb.Dataset(x_train, label=y_train)

# Setando os parâmetros do LightGBM
parameters_lgbm = {'num_leaves':150,   #número de folhas em uma árvore; padrão = 31; type = int
              'objective':'binary',
              'max_depth':7, # Aqui definimos max_profundidade em XGB e LightGBM como 7 para ter uma comparação justa entre os dois.
              'learning_rate':.05,
              'max_bin':200}

# Setando as métricas de avaliação durante o treinamento do modelo.
parameters_lgbm['metric'] = ['auc', 'binary_logloss']

num_round=50
start=datetime.now()

# Treinando o modelo
lgbm=lgb.train(parameters_lgbm, train_data_lgb, num_round)
stop=datetime.now()

# Executando o modelo
execution_time_lgbm = stop-start
execution_time_lgbm

print("Tempo de execução: ", execution_time_lgbm)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5487, number of negative: 17305
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017975 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 76
[LightGBM] [Info] Number of data points in the train set: 22792, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240742 -> initscore=-1.148614
[LightGBM] [Info] Start training from score -1.148614
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [54]:
# Predição do teste
ypred_lgbm = lgbm.predict(x_test)
ypred_lgbm[0:5]  # Apresentando as 5 primeiras predições

array([0.04298613, 0.04178775, 0.49599265, 0.38488225, 0.63265908])

In [55]:
# Convert predictions to binary using a threshold (e.g., 0.5)
ypred_lgbm = (ypred_lgbm >= 0.5).astype(int)

In [56]:
# Calculando a Acurácia
accuracy_lgbm = accuracy_score(ypred_lgbm, y_test)
accuracy_lgbm
print(f"Acurácia: {accuracy_lgbm * 100:.2f}%")

Acurácia: 82.20%


In [57]:
y_test.value_counts()

,count
salary,
0,7415
1,2354


In [58]:
# Calculando o roc_auc_score do LGBM
auc_lgbm = roc_auc_score(y_test, ypred_lgbm)
auc_lgbm

print(f"ROC Acurácia: {auc_lgbm * 100:.2f}%")



ROC Acurácia: 69.67%


# **# Avaliação: XGBoost vs Light_GBM**

O **roc_auc_score** é uma métrica única que resume o desempenho de um modelo de classificação binária. Ele representa a capacidade do modelo de distinguir entre as duas classes. Quanto maior o valor do roc_auc_score, melhor o modelo em classificar corretamente os exemplos.

In [59]:
#Criando o dataframe ‘comparison_df’ para comparar a performance LightGBM e XGBoost.
comparison_dict = {'Accuracy Score':(f"{accuracy_lgbm * 100:.2f}%", f"{accuracy_xgb * 100:.2f}%"),
                   'AUC Score':(f"{auc_lgbm * 100:.2f}%", f"{auc_xgb * 100:.2f}%"),
                   'Execution Time':(execution_time_lgbm, execution_time_xgb)}

comparison_df = DataFrame(comparison_dict)
comparison_df.index = ['LightGBM','XGBoost']
comparison_df


,Accuracy Score,AUC Score,Execution Time
LightGBM,82.20%,69.67%,0 days 00:00:00.318086
XGBoost,82.30%,69.92%,0 days 00:00:00.456402


**Conclusão:**

No geral, a execução do código foi bem-sucedida, com a construção de duas máquinas preditivas com bom desempenho. O **LightGBM** se destacou por ser mais rápido e apresentar resultados ligeiramente melhores. No entanto, há espaço para aprimoramentos, como a exploração de hiperparâmetros, análise de erros e interpretabilidade do modelo.